In [1]:
import asyncio
import json
from datetime import datetime
import requests
import hashlib
from elasticsearch import Elasticsearch

import requests
import web3
import pandas as pd

import sys
sys.path.append("..")

import libs.common.utils as utils
import libs.aave.api as aave_api
import libs.common.payload as payload
import libs.common.etherscan as etherscan
import eth_abi
import config.config as cfg

import warnings
warnings.filterwarnings('ignore')

In [2]:
es = Elasticsearch(
    "https://localhost:9200",
    http_auth=('elastic', 'y=fUp=8ucKL18I5K=1Am'),
    verify_certs=False
)

In [3]:
def es_create_index(index_name):
    try:
        es.indices.create(index=index_name)
    except Exception as e:
        print(e)

In [4]:
index_name = 'abi'
es_create_index(index_name)
# es.indices.delete(index=index_name, ignore=[400, 404])

BadRequestError(400, 'resource_already_exists_exception', 'index [abi/G6D1tWcVQ8ichz91d0BLUg] already exists')


In [5]:
# use aave v2 as an example
ws = web3.Web3(web3.Web3.HTTPProvider(cfg.config["http_url"]))

In [6]:
def doc_id(contract_addr, abi):
    return hashlib.md5('{}/{}'.format(contract_addr,str(abi)).encode()).hexdigest()

In [7]:
def add_abi(contract_addr, impl_addr, contract_name, author = "rshi"):
    # if contract addr is different from impl addr
    pool_contract = etherscan.get_contract(contract_addr,impl_addr)
    # save contract abi
    doc = {
        'author': 'rshi',
        'timestamp': datetime.now(),
        'id' : doc_id(contract_addr, pool_contract.abi),
        'address' : contract_addr,
        'contract' : contract_name,
        'type' : "contract",
        'name' : contract_name,
        'abi': pool_contract.abi
    }
    resp = es.index(index='abi', id=doc['id'], document=doc)

    # save contract function and event abi
    for abi in pool_contract.abi:
        if abi["type"] != "constructor":
            doc = {
                'author': 'rshi',
                'timestamp': datetime.now(),
                'id' : doc_id(contract_addr, abi),
                'address' : contract_addr,
                'contract' : contract_name,
                'type' : abi["type"],
                'name' : abi["name"],
                'abi': abi
            }
            resp = es.index(index='abi', id=doc['id'], document=doc)

In [8]:
contract_addr = "0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7"
impl_addr = contract_addr
contract_name = "Curve.fi"
add_abi(contract_addr, impl_addr, contract_name)

In [9]:
contract_addr = "0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9"
impl_addr = "0x085E34722e04567Df9E6d2c32e82fd74f3342e79"
contract_name = "Aave v2"
add_abi(contract_addr, impl_addr, contract_name)